### Importamos librerias

In [13]:
#%pip install beautifulsoup4 lxml
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests

### Buscamos en Wikipedia

In [14]:
url = 'https://es.wikipedia.org/wiki/Sierra_Norte_(Madrid)'
response = requests.get(url)
response

<Response [200]>

### Pasamos la pagina a BeautifulSoup

In [15]:
pagina = bs(response.content, "html.parser")

### Separamos la comarca por subcomarcas y municipios

In [16]:
subcomarcas = {} # Diccionario para almacenar las subcomarcas
for text in pagina.select('div.thumbcaption span'):
    try:
        slicers = [text.text.strip('\xa0 ').index("("), text.text.strip('\xa0').index(':')] # Estos indices nos ayudarán a hacer slicing y eliminar caracteres no deseados
        subcomarca = (text.text.strip('\xa0 ')[:slicers[0]-1])
        subcomarcas[subcomarca] = (text.text.strip('\xa0 ')[slicers[1]+1:-1]).split('; ')
    except:
        pass
# Diccionario completo con {'Subcomarca' : [Municipios]}
#print(subcomarcas)

# Creamos el Dataframe de las subcomarcas
df_subcomarcas = pd.DataFrame.from_dict(subcomarcas, orient='index').T

# Lo guardamos como csv
df_subcomarcas.to_csv('./csv/Subcomarcas.csv', encoding='latin1')

---

### Ahora vamos a hacer scrapping para obtener la poblacion y la superficie por municipio

In [17]:
# Elemento que contiene la tabla
table = pagina.find('table', class_='wikitable sortable')

# Pasamos la tabla a dataframe
df_habitantes = pd.read_html(str(table), thousands='.', decimal= ',')[0]

# Tenemos que preparar y quitar errores que haya en el dataframe
df_habitantes['Población'] = df_habitantes['Población'].str.split().str.join('')

# Columna densidad de poblacion por municipio
df_habitantes['Densidad'] = round(df_habitantes['Población'].astype(np.number) / df_habitantes['Superficie'].astype(np.number), 2)

# Lo guardamos como csv
df_habitantes.to_csv('./csv/Habitantes.csv', encoding='latin1')

---

### Mergeamos toda la información obtenida

In [18]:
# Esta funcion nos permite encontrar a que subcomarca pertenece cada municipio
def subcomarca_municipio(municipio, dic):
    for key, valor in dic.items():
        if municipio in valor:
            return key

In [19]:
# Creamos el DataFrame final
df_final = df_habitantes.copy()
df_final.insert(0, 'Subcomarca', df_habitantes['Municipio'].apply(subcomarca_municipio, args= [subcomarcas]))


In [20]:
# Comprobamos si algo ha fallado
print(df_final.isna().sum()) # Vemos que hay 2 null en Subcomarca
print(df_final[df_final.isnull().any(axis=1)]) # No se ha imputado bien al no tener el nombre igual, lo hacemos manualmente
row = df_final.index[df_final['Municipio'] == 'Horcajo de la Sierra-Aoslos'] # Sacamos el indice (No hace falta pero queda más profesional, supongo...)
df_final.loc[row, 'Subcomarca'] = 'Valle Medio del Lozoya'

Subcomarca    2
Municipio     0
Superficie    0
Población     0
Densidad      0
dtype: int64
   Subcomarca                    Municipio  Superficie Población  Densidad
0        None                Total comarca     1252.89     31930     25.49
15       None  Horcajo de la Sierra-Aoslos       20.57       170      8.26


In [21]:
# Ponemos bonito el dataframe
df_final.rename(columns = {'Superficie': 'Superficie (Km²)'}, inplace=True)
df_final.sort_values('Subcomarca').reset_index(drop=True)

# Lo guardamos
df_final.to_csv('./csv/Pre.csv', encoding='latin1')